<a href="https://colab.research.google.com/github/rahulsing/pytorch_demo/blob/master/07_RNN_Gender_By_Names_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import pandas as pd

names_data=pd.read_csv("/content/gdrive/My Drive/datasets/gender_by_names/names.csv")
names_data.head()

,Name,Gender
0,Aamir,Male
1,Aaron,Female
2,Aaron,Male
3,Abagael,Female
4,Abagail,Female


In [4]:
len(names_data)

10132

In [5]:
len(names_data['Name'].unique())

9543

In [0]:
import random

names_data=names_data.drop_duplicates(subset=['Name'],keep=random.choice(['first','last']))

In [7]:
len(names_data)

9543

In [10]:
#https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
 
from sklearn import preprocessing
le=preprocessing.LabelEncoder()  
names_data['Gender']=le.fit_transform(names_data['Gender'])
names_data.head()

,Name,Gender
0,Aamir,1
1,Aaron,0
3,Abagael,0
4,Abagail,0
5,Abbe,0


In [0]:
genders=['Female','Male']

In [0]:
import string
all_letters=string.ascii_letters+".,;'"

In [14]:
n_letters=len(all_letters)
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,;'"

In [0]:
import torch
def name_to_tensor(name):
  name_in_tensor=torch.zeros(len(name),1,n_letters)
  for i,letter in enumerate(name):
    name_in_tensor[i][0][all_letters.find(letter)]=1
    
  return name_in_tensor

In [20]:
name_to_tensor('a')

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]]])

In [23]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

    100% |████████████████████████████████| 91.1MB 114.9MB/s 
    100% |████████████████████████████████| 2.0MB 10.8MB/s 
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.


In [21]:
name_to_tensor('aA')

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]]])

In [0]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(RNN,self).__init__()
    
    self.hidden_size=hidden_size
    
    self.i2h=nn.Linear(input_size+hidden_size,hidden_size)
    self.i2o=nn.Linear(input_size+hidden_size,output_size)
    self.softmax=nn.LogSoftmax(dim=1)
    
  def forward(self,input,hidden):
    combined=torch.cat((input,hidden),1)
    hidden=self.i2h(combined)
    output=self.i2o(combined)
    output=self.softmax(output)
    
    return output,hidden
  
  def initHidden(self):
    return torch.zero(1,self.hidden_size)

In [0]:
n_hidden=128
n_genders=len(genders)

rnn=RNN(n_letters,n_hidden,output_size=n_genders)

In [0]:
iterations=100000

criterion=nn.NLLLoss()

learning_rate=0.005

In [0]:
def output_to_gender(output):
  top_n,top_index=output.topk()
  pred_i=top_index(0).item()
  
  pred=genders[pred_i]
  
  return pred